<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/bert_replace/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount GDriver
Perpare Colab environment.

In [0]:
locale = False
try:
    import google.colab
    google.colab.drive.mount('/gdrive')
except ModuleNotFoundError:
    locale = True

In [0]:
addh = "/gdrive/My Drive"
if locale:
    addh = "/mnt/d/My Drive"
    print("[Locale] Using address head: " + addh)
else:
    print("[Colab] Using address head: " + addh)

# Prepare Input

In [0]:
import preprocess

In [0]:
train_char_seqs, train_tag_seqs = preprocess.load_file(addh + "/Graduation/Data/train_dict.txt")
test_char_seqs, test_tag_seqs = preprocess.load_file(addh + "/Graduation/Data/test_dict.txt")
train_token_id_seqs, train_tag_id_seqs, tag_vocab = preprocess.preprocess(train_char_seqs, train_tag_seqs)
test_token_id_seqs, test_tag_id_seqs, tag_vocab = preprocess.preprocess(test_char_seqs, test_tag_seqs)

In [0]:
train_token_id_seqs[0]

In [0]:
train_tag_id_seqs[0]

In [0]:
tag_vocab

# Build Model
using BERT

In [0]:
import keras
import keras_bert 

In [0]:
bert_model = keras_bert.load_trained_model_from_checkpoint(
    # config_file
    addh + "/Graduation/BERT/multi_cased_L-12_H-768_A-12/bert_config.json", 
    # checkpoint file
    addh + "/Graduation/BERT/multi_cased_L-12_H-768_A-12/bert_model.ckpt", 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True

)

In [0]:
MAX_LEN = 356

input_token = keras.layers.Input(shape=(MAX_LEN,))
input_segment = keras.layers.Input(shape=(MAX_LEN,))

bert_output = bert_model([input_token, input_segment])

output = keras.layers.Dense(16, activation=keras.activations.softmax)(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [0]:
model.compute_output_shape([(1, 5), (1, 5)])

In [0]:
bert_model.summary()

In [0]:
model.summary()

# Fine-tuning

In [0]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00004),
    loss=keras.losses.categorical_crossentropy,
    metrics=["accuracy"]
)

In [0]:
model.fit(
    x=train_token_id_seqs, 
    y=train_tag_id_seqs, 
    batch_size=16, 
    epochs=10,
    verbose=2,
    validation_data=(test_token_id_seqs, test_tag_id_seqs),
    shuffle=True
)